In [1]:
import os

In [2]:
%pwd

'd:\\college_work\\4th year\\Sem7th\\Project\\OBE\\Paper- Blooms taxanomy\\Boolm-s-Level-Detection-A-MLOPS-Project\\research'

In [3]:
os.chdir("../")
%pwd

'd:\\college_work\\4th year\\Sem7th\\Project\\OBE\\Paper- Blooms taxanomy\\Boolm-s-Level-Detection-A-MLOPS-Project'

## testing and researching for model

In [5]:
import pandas as pd

In [ ]:
df = pd.read_csv("./artifacts/data_transformation/train.csv")
df

### Model 1 NB claassifier

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, classification_report

train_data = pd.read_csv("./artifacts/data_transformation/train.csv")
test_data = pd.read_csv("./artifacts/data_transformation/test.csv")

train_x = train_data["Text"]
train_y = train_data["Level"]
test_x = test_data["Text"]
test_y = test_data["Level"]

model_Nb = Pipeline(
    [
        ("vectorizer", TfidfVectorizer()),
        ("SMOTE ", SMOTE(random_state=12)),
        ("MultinomialNB", MultinomialNB()),
    ]
)

model_Nb.fit(train_x, train_y)
model_Nb_predit = model_Nb.predict(test_x)
# 
# print(model_Nb_predit)

# print(classification_report(test_y, model_Nb_predit))
print(f"accuracy {accuracy_score(test_y,model_Nb_predit)}")

accuracy 0.9261083743842364


In [17]:
model_Nb_predit = model_Nb.predict(["What are our organization’s core values?"])
model_Nb_predit

array(['Remember'], dtype='<U10')

### model 2 distil bert classification

In [ ]:
%pip install tensorflow==2.15.0
%pip install --upgrade tensorflow

In [ ]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification

In [ ]:
url = f"https://ukantjadia.me/bt/train.csv"
df = pd.read_csv(url)

In [ ]:
df['count'] = df['Text'].apply(lambda x: len(x.split()))
plt.figure(figsize=(10,6))

sns.displot(df['count'])
plt.xlabel('The no of words')
plt.ylabel('no of words distribution ')


In [ ]:
category_count = df['Level'].value_counts()
categories = category_count.index
categories

In [ ]:
fig = plt.figure(figsize= (12, 5))
ax = fig.add_subplot(111)
sns.barplot(x = category_count.index, y = category_count )
for a, p in enumerate(ax.patches):
    ax.annotate(f'{categories[a]}\n' + format(p.get_height(), '.0f'), xy = (p.get_x() + p.get_width() / 2.0, p.get_height()), xytext = (0,-25), size = 13, color = 'white' , ha = 'center', va = 'center', textcoords = 'offset points', bbox = dict(boxstyle = 'round', facecolor='none',edgecolor='white', alpha = 0.5) )

plt.xlabel('Categories', size = 15)
plt.ylabel('The Number of News', size= 15)
plt.xticks(size = 12)
plt.title("The number of News by Categories" , size = 18)
plt.show()

In [ ]:
df['encoded_text'] = df['Level'].astype('category').cat.codes
df.head(10)

In [ ]:
data_texts = df['Text'].to_list()
data_labels = df['encoded_text'].to_list()

train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels,test_size = 0.2,random_state = 0)
train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels,test_size = 0.01,random_state = 0)

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=6)
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',
    num_train_epochs=7,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=1e-5,
    logging_dir='./logs',
    eval_steps=100
)

with training_args.strategy.scope():
    trainer_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels = 6)

trainer = TFTrainer(
    model = trainer_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [ ]:
trainer.train()
trainer.evaluate()

In [ ]:
save_directory = "saved_models"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

In [ ]:
tokenizer_fine_tuned = DistilBertTokenizer.from_pretrained(save_directory)

model_fine_tuned = TFDistilBertForSequenceClassification.from_pretrained(save_directory)

In [ ]:
test_text = "How wdfsdfsf this type of emergency?"
predict_input = tokenizer_fine_tuned.encode(
    test_text,
    truncation = True,
    padding = True,
    return_tensors = 'tf'
)

output = model_fine_tuned(predict_input)[0]

prediction_value = tf.argmax(output, axis = 1).numpy()[0]

prediction_value


### model 3 word to vect

In [ ]:
import gensim.downloader as api
word_vect = api.load('word2vec-google-news-300')



In [ ]:
import spacy
import numpy as np
!python -m spacy download en_core_web_md
nlp = spacy.load('en_core_web_md')

In [13]:
# tokenises a sentence using spacy library
def spacy_tokenizer(text):
    doc = nlp(text)
    # lemmatise tokens
    tokens = [word.lemma_.strip() for word in doc]
    return tokens

def sentence_vect(tokens):
    size = word_vect.vector_size
    sent_vect = np.zeros(size)
    counter = 0
    for word in tokens:
        if word in word_vect:
            sent_vect += word_vect[word]
            counter += 1

    if counter == 0:
        return None
    sent_vect = sent_vect / counter
    return sent_vect

In [ ]:
import nltk
nltk.download('punkt')
import os
from itertools import islice
from nltk.tokenize import word_tokenize,sent_tokenize

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

url = f"https://ukantjadia.me/bt/train.csv"

df = pd.read_csv(url)

df['new_Tokens'] = df['Text'].apply(spacy_tokenizer)
# create sentence vectors by taking average of token vectors
df['Vectors'] = df['new_Tokens'].apply(sentence_vect)
df = df.dropna(axis=0, subset=['Vectors'])
train, test = train_test_split(df, test_size=0.2)

In [ ]:
from sklearn.svm import SVC

model = SVC(kernel='linear', gamma='auto', degree=3)
categories = ['Remember', 'Understand', 'Apply', 'Analyse', 'Evaluate', 'Create']
model.fit(list(train.Vectors), train.Level)
labels = model.predict(list(test.Vectors))
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
mat = confusion_matrix(test.Level, labels)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False
            , xticklabels=categories
            , yticklabels=categories)
plt.xlabel('true Level')
plt.ylabel('predicted Level')

print(classification_report(test.Level, labels))
acc = accuracy_score(test.Level, labels)
print("Accuracy:", acc)

In [ ]:
category_dict = {0: 'Remember', 1: 'Understand', 2: 'Apply', 3: 'Analyse', 4: 'Evaluate', 5: 'Crexcate'}

def predict_blooms(text, model):
    process = spacy_tokenizer(text)
    processes = sentence_vect(process)
    blooms = model.predict([processes])
    print("Predicted Class:", blooms)
    
print(categories, "\n")

task = "What changes would you make to this example contract?"

print("\nSVM PREDICTION:")
predict_blooms(task, model)

## Standard code

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    target_columns: str
    training_columns: str
    pipeline_ele_1: str
    pipeline_ele_2: str
    pipeline_ele_3: str
    elastic_model_name: str
    multinomial_model_name: str
    word2vector_model_name: str
    distilBERT_model_name: str

In [5]:
from blooms.constants import *
from blooms.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_training_config(self) -> ModelTrainingConfig:
        config = self.config.model_training
        params = self.params.Models_03
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        model_training_config = ModelTrainingConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            target_columns=schema.target_column,
            training_columns=schema.training_columns,
            elastic_model_name=config.elastic_model_name,
            multinomial_model_name=config.multinomial_model_name,
            distilBERT_model_name=config.distilBERT_model_name,
            word2vector_model_name=config.word2vector_model_name,
            pipeline_ele_1=params.pipeline_ele_1,
            pipeline_ele_2=params.pipeline_ele_2,
            pipeline_ele_3=params.pipeline_ele_3,
        )
        return model_training_config

In [6]:
from blooms import logger
from sklearn.linear_model import ElasticNet
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.naive_bayes import MultinomialNB

import pandas as pd
import joblib
import os

In [7]:
class ModelTrainer:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        train_x = train_data[self.config.training_columns]
        train_y = train_data[[self.config.target_columns]]

        print(f"this is the train x size {train_x.head(2)}")
        print(f"this is the train y size {train_y.head(2)}")
        # elasticNet = ElasticNet(
        #     alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=34
        # )
        # elasticNet.fit(train_x, train_y)
        # joblib.dump(elasticNet, os.path.join(self.config.root_dir, self.config.el))

        multiNomial = Pipeline(
            [
                (self.config.pipeline_ele_1, TfidfVectorizer()),
                (self.config.pipeline_ele_2, SMOTE(random_state=12)),
                (self.config.pipeline_ele_3, MultinomialNB()),
            ]
        )
        multiNomial.fit(train_x, train_y)
        joblib.dump(
            multiNomial,
            os.path.join(self.config.root_dir, self.config.multinomial_model_name),
        )
        logger.info(f"Model training done! Model saved at {self.config.multinomial_model_name}")

In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_training_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2023-12-19 11:01:45: INFO: common: yaml file: config\config.yaml loaded succcessfully]
[2023-12-19 11:01:45: INFO: common: yaml file: params.yaml loaded succcessfully]
[2023-12-19 11:01:45: INFO: common: yaml file: schema.yaml loaded succcessfully]
[2023-12-19 11:01:45: INFO: common: created directory at : artifacts]
[2023-12-19 11:01:45: INFO: common: created directory at : artifacts/model_training]
this is the train x size 0    list are four special types of personal clause...
1    prepare a special book jacket binding that acc...
Name: Text, dtype: object
this is the train y size       Level
0  Remember
1    Create
[2023-12-19 11:01:46: INFO: 3807993266: Model training done! Model saved at multinomial_model.joblib]


c:\Users\Ukant\anaconda3\envs\blooms\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
